In [4]:
time_start = 20150101
time_end =  20210630
today = 20211130
feat_list = ['pre_close','open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover','chg']
return_value = '5d_ret'
output_path = '/home/wuwenjun/Data/AlphaNet_Original_Input/'
target = '5d_ret'
day_back = 5 # 回溯几天进行计算
day = 30 # 一次提取几天的数,一般默认为30
stride = 10 # 一次学习多少天

In [5]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm

In [3]:
dataframe_list = DataAPI.get_stock_prices(start_date=time_start, end_date=time_end).reset_index().set_index(['timestamp','ticker'])

In [4]:
dataframe_list = dataframe_list[feat_list].reset_index()
time_df = pd.DataFrame(dataframe_list['timestamp'].drop_duplicates())
time_df['timestamp_shift'] = time_df['timestamp'].apply(lambda x : DataAPI.get_next_trading_day(x, output='datetime'))
dataframe_list = pd.merge(left = dataframe_list , right = time_df , how = 'left',on = 'timestamp')
dataframe_list.drop(['timestamp'],axis=1,inplace=True)
dataframe_list.rename(columns = {"timestamp_shift":"timestamp"},inplace=True)
dataframe_list['timestamp'] = pd.to_datetime(dataframe_list['timestamp']) + pd.Timedelta(hours=9,minutes = 30)
dataframe_list.set_index(['timestamp','ticker'],inplace = True)
dataframe_list

pre_close    open    high     low   close  \
timestamp           ticker                                              
2015-01-06 09:30:00 000001      15.84   15.99   16.28   15.60   16.02   
                    000002      13.90   14.39   15.29   14.22   14.91   
                    000004      15.57   15.58   15.90   15.30   15.69   
                    000005       4.10    4.10    4.10    4.10    4.10   
                    000006       7.05    7.10    7.39    7.02    7.08   
...                               ...     ...     ...     ...     ...   
2021-07-01 09:30:00 688700      46.60   47.80   54.50   47.03   53.38   
                    688777      90.00   89.95   94.80   89.06   94.15   
                    688788     138.20  137.28  148.50  136.33  145.38   
                    688819      43.68   43.40   44.98   43.31   44.49   
                    688981      59.25   59.69   62.50   59.40   61.82   

                                vwap      volume  pct_chg  turnover  \
timestamp           ticker                                            
2015-01-06 09:30:00 000001   15.9605  2860436.43   1.1364    2.9079   
                    000002   14.7858  6560835.70   7.2662    6.7834   
                    000004   15.5709    28095.12   0.7707    3.3496   
                    000005    4.1000        0.00   0.0000    0.0000   
                    000006    7.1881  1078647.12   0.4255    8.0562   
...                              ...         ...      ...       ...   
2021-07-01 09:30:00 688700   50.7153    64336.83  14.5494   19.2222   
                    688777   92.9798    19001.61   4.6111    4.6288   
                    688788  144.3683    13572.56   5.1954    7.7977   
                    688819   44.3727    47249.01   1.8544    4.6754   
                    688981   61.5420   777950.53   4.3376    7.1003   

                            free_turnover   chg  
timestamp           ticker                       
2015-01-06 09:30:00 000001         5.7657  0.18  
                    000002         8.1741  1.01  
                    000004         4.5343  0.12  
                    000005         0.0000  0.00  
                    000006        13.2751  0.03  
...                                   ...   ...  
2021-07-01 09:30:00 688700        19.2222  6.78  
                    688777         4.6288  4.15  
                    688788         7.7977  7.18  
                    688819         4.6754  0.81  
                    688981         7.1003  2.57  

[5357761 rows x 11 columns]

In [5]:
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/ShareFolder/factor_lib")

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Process ForkPoolWorker-141:
Process ForkPoolWorker-131:
Process ForkPoolWorker-130:
Process ForkPoolWorker-142:
Process ForkPoolWorker-140:
Process ForkPoolWorker-75:
Process ForkPoolWorker-91:
Process ForkPoolWorker-74:
Process ForkPoolWorker-86:
Process ForkPoolWorker-137:
Process ForkPoolWorker-128:
Process ForkPoolWorker-133:
Process ForkPoolWorker-144:
Process ForkPoolWorker-143:
Process ForkPoolWorker-100:
Process ForkPoolWorker-94:
Process ForkPoolWorker-104:
Process ForkPoolWorker-124:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-88:
Process ForkPoolWorker-80:
Traceback (most recent call last):
Process ForkPoolWorker-119:
Process ForkPoolWorker-139:
Process ForkPoolWorker-103:
Process ForkPoolWorker-97:
Process ForkPoolWorker-132:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call 

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  

In [6]:
FT.load_feature_from_dataframe(dataframe_list,universe='Investable')
FT.load_return_data()

Merging feature and return...: 100%|█████████████████████████████████████████████████████████████████████████████| 1580/1580 [00:08<00:00, 195.78it/s]


In [7]:
data_info = FT.data_info

In [8]:
# Drop 掉不需要的return 列
df = pd.DataFrame()
return_list = feat_list.copy()
return_list.append(return_value)
for timestamp in tqdm(data_info.keys()):
    timestamp = pd.to_datetime(timestamp)
    one_data = data_info[timestamp][return_list].reset_index()
    one_data['timestamp'] = timestamp
    df = pd.concat([df,one_data],axis=0)
df.sort_values(by = ['timestamp','ticker'],inplace=True)
display(df)

100%|██████████| 1580/1580 [05:40<00:00,  4.65it/s]


,ticker,pre_close,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,chg,5d_ret,timestamp
0,000001,15.84,15.99,16.28,15.60,16.02,15.9605,2860436.43,1.1364,2.9079,5.7657,0.18,-0.075710,2015-01-06 09:30:00
1,000002,13.90,14.39,15.29,14.22,14.91,14.7858,6560835.70,7.2662,6.7834,8.1741,1.01,-0.106164,2015-01-06 09:30:00
2,000004,15.57,15.58,15.90,15.30,15.69,15.5709,28095.12,0.7707,3.3496,4.5343,0.12,0.006390,2015-01-06 09:30:00
3,000006,7.05,7.10,7.39,7.02,7.08,7.1881,1078647.12,0.4255,8.0562,13.2751,0.03,-0.078571,2015-01-06 09:30:00
4,000007,14.60,14.51,14.80,14.25,14.52,14.5105,28415.72,-0.5479,1.3807,1.7538,-0.08,-0.004844,2015-01-06 09:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4024,688699,237.77,285.32,285.32,285.32,285.32,285.3200,1225.82,19.9983,0.6940,0.6940,47.55,-0.024476,2021-07-01 09:30:00
4025,688777,90.00,89.95,94.80,89.06,94.15,92.9798,19001.61,4.6111,4.6288,4.6288,4.15,0.053133,2021-07-01 09:30:00
4026,688788,138.20,137.28,148.50,136.33,145.38,144.3683,13572.56,5.1954,7.7977,7.7977,7.18,-0.102721,2021-07-01 09:30:00
4027,688819,43.68,43.40,44.98,43.31,44.49,44.3727,47249.01,1.8544,4.6754,4.6754,0.81,0.032164,2021-07-01 09:30:00


In [ ]:
df.set_index(['timestamp','ticker']).to_csv(output_path + 'data.csv')
# Save_TimeStamp
df['timestamp'].drop_duplicates()

In [23]:
df_no_return
for i in range(1,2):
    display(df_no_return.shift(-i))

pre_close    open    high     low   close  \
timestamp           ticker                                              
2015-01-06 09:30:00 1           13.90   14.39   15.29   14.22   14.91   
                    2           15.57   15.58   15.90   15.30   15.69   
                    4            7.05    7.10    7.39    7.02    7.08   
                    6           14.60   14.51   14.80   14.25   14.52   
                    7           22.09   22.10   23.03   21.60   21.70   
...                               ...     ...     ...     ...     ...   
2021-07-01 09:30:00 688699      90.00   89.95   94.80   89.06   94.15   
                    688777     138.20  137.28  148.50  136.33  145.38   
                    688788      43.68   43.40   44.98   43.31   44.49   
                    688819      59.25   59.69   62.50   59.40   61.82   
                    688981        NaN     NaN     NaN     NaN     NaN   

                                vwap      volume  pct_chg  turnover  \
timestamp           ticker                                            
2015-01-06 09:30:00 1        14.7858  6560835.70   7.2662    6.7834   
                    2        15.5709    28095.12   0.7707    3.3496   
                    4         7.1881  1078647.12   0.4255    8.0562   
                    6        14.5105    28415.72  -0.5479    1.3807   
                    7        22.3283    22900.58  -1.7655    1.5546   
...                              ...         ...      ...       ...   
2021-07-01 09:30:00 688699   92.9798    19001.61   4.6111    4.6288   
                    688777  144.3683    13572.56   5.1954    7.7977   
                    688788   44.3727    47249.01   1.8544    4.6754   
                    688819   61.5420   777950.53   4.3376    7.1003   
                    688981       NaN         NaN      NaN       NaN   

                            free_turnover   chg  
timestamp           ticker                       
2015-01-06 09:30:00 1              8.1741  1.01  
                    2              4.5343  0.12  
                    4             13.2751  0.03  
                    6              1.7538 -0.08  
                    7              1.7862 -0.39  
...                                   ...   ...  
2021-07-01 09:30:00 688699         4.6288  4.15  
                    688777         7.7977  7.18  
                    688788         4.6754  0.81  
                    688819         7.1003  2.57  
                    688981            NaN   NaN  

[4851238 rows x 11 columns]

In [6]:
df = pd.read_csv("/home/wuwenjun/Data/AlphaNet_Original_Input.csv")
df

,timestamp,ticker,pre_close,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,chg,5d_ret
0,2015-01-06 09:30:00,1,15.84,15.99,16.28,15.60,16.02,15.9605,2860436.43,1.1364,2.9079,5.7657,0.18,-0.075710
1,2015-01-06 09:30:00,2,13.90,14.39,15.29,14.22,14.91,14.7858,6560835.70,7.2662,6.7834,8.1741,1.01,-0.106164
2,2015-01-06 09:30:00,4,15.57,15.58,15.90,15.30,15.69,15.5709,28095.12,0.7707,3.3496,4.5343,0.12,0.006390
3,2015-01-06 09:30:00,6,7.05,7.10,7.39,7.02,7.08,7.1881,1078647.12,0.4255,8.0562,13.2751,0.03,-0.078571
4,2015-01-06 09:30:00,7,14.60,14.51,14.80,14.25,14.52,14.5105,28415.72,-0.5479,1.3807,1.7538,-0.08,-0.004844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4851233,2021-07-01 09:30:00,688699,237.77,285.32,285.32,285.32,285.32,285.3200,1225.82,19.9983,0.6940,0.6940,47.55,-0.024476
4851234,2021-07-01 09:30:00,688777,90.00,89.95,94.80,89.06,94.15,92.9798,19001.61,4.6111,4.6288,4.6288,4.15,0.053133
4851235,2021-07-01 09:30:00,688788,138.20,137.28,148.50,136.33,145.38,144.3683,13572.56,5.1954,7.7977,7.7977,7.18,-0.102721
4851236,2021-07-01 09:30:00,688819,43.68,43.40,44.98,43.31,44.49,44.3727,47249.01,1.8544,4.6754,4.6754,0.81,0.032164


In [7]:
# for ticker in tqdm(df['ticker'].drop_duplicates()):
def test(ticker):
    one_data = df[df['ticker'] == ticker]
    one_data.set_index(['timestamp','ticker'],inplace=True)
    target_df = one_data[target]
    one_data.drop(target,axis=1,inplace=True)
    concat_df = pd.DataFrame()
    for day in range(29,-1,-1):
        concat_df = pd.concat([concat_df,one_data.shift(day)],axis=1)
    concat_df.dropna(axis=0,how="any",inplace=True)
    concat_df = pd.concat([concat_df,target_df],axis=1,join='inner')
#     concat_df.to_csv(output_path + str(ticker) +".csv")
    return concat_df

In [8]:
from multiprocessing import Pool
import multiprocessing as mp
import time
if __name__=="__main__":
    num_cores = int(mp.cpu_count())
    pool = mp.Pool(num_cores)
    result = {}
    t1 = time.time()
    for ticker in tqdm(df['ticker'].drop_duplicates()):
        result[ticker] = (pool.apply_async(test, args=(ticker,)))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
    pool.close()
    pool.join()
    t2 = time.time()
    print("并行执行时间：", int(t2 - t1))
    df_list = []
    for i in tqdm(result):
        df_list.append(result[i].get())
    final_df = pd.concat(df_list)

100%|██████████| 4276/4276 [00:00<00:00, 126769.51it/s]

并行执行时间： 94


In [9]:
final_df.reset_index(inplace=True)
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])
final_df.columns = [str(i) for i in range(final_df.shape[1])]
final_df.rename(columns={'0': 'timestamp', '1': 'ticker', str(final_df.shape[1] - 1):'target'}, inplace=True)
final_df

,timestamp,ticker,2,3,4,5,6,7,8,9,...,323,324,325,326,327,328,329,330,331,target
0,2015-02-16 09:30:00,1,15.84,15.99,16.28,15.60,16.02,15.9605,2860436.43,1.1364,...,14.18,13.84,13.95,14.0189,887743.10,0.6494,0.9025,1.7894,0.09,0.005735
1,2015-02-17 09:30:00,1,16.02,15.85,16.39,15.55,15.78,15.9408,2166421.40,-1.4981,...,14.01,13.73,13.92,13.8643,721202.39,-0.2151,0.7332,1.4537,-0.03,0.001433
2,2015-02-25 09:30:00,1,15.78,15.56,15.83,15.30,15.48,15.4977,1700120.67,-1.9011,...,14.11,13.93,13.99,14.0256,680491.59,0.5029,0.6918,1.3717,0.07,-0.029915
3,2015-02-26 09:30:00,1,15.48,15.50,15.57,14.90,14.96,15.1167,1407714.21,-3.3592,...,14.05,13.76,13.80,13.8414,577392.33,-1.3581,0.5870,1.1638,-0.19,-0.021739
4,2015-02-27 09:30:00,1,14.96,14.90,15.87,14.71,15.08,15.2895,2508500.23,0.8021,...,14.08,13.60,14.07,13.8859,1025544.24,1.9565,1.0426,2.0672,0.27,-0.049075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4728009,2021-06-25 09:30:00,601399,4.23,4.10,4.11,3.96,3.96,4.0407,504073.25,-6.3830,...,3.39,3.34,3.36,3.3594,154445.60,-1.1765,0.6496,0.6496,-0.04,-0.011940
4728010,2021-06-28 09:30:00,601399,3.96,3.95,3.99,3.82,3.83,3.8798,388797.28,-3.2828,...,3.45,3.35,3.41,3.4084,200886.20,1.4881,0.8450,0.8450,0.05,-0.052326
4728011,2021-06-29 09:30:00,601399,3.83,3.80,3.83,3.66,3.67,3.7162,307810.00,-4.1775,...,3.44,3.37,3.38,3.3942,144365.21,-0.8798,0.6072,0.6072,-0.03,-0.023810
4728012,2021-06-30 09:30:00,601399,3.67,3.67,3.78,3.66,3.72,3.7227,208419.16,1.3624,...,3.40,3.35,3.36,3.3747,103839.37,-0.5917,0.4368,0.4368,-0.02,-0.026786


In [10]:
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
time_list = [20140101,20180101,20180630,20181231,,20190630,20191231,20200601,20201231,20210630]

for i in range(len(time_list)-1):
    start_time = pd.to_datetime(str(time_list[i]))
    end_time = pd.to_datetime(str(time_list[i+1]))
    final_df[
        (final_df['timestamp'] >= start_time) & 
        (final_df['timestamp'] < end_time)].set_index(['timestamp','ticker']).to_parquet(
        output_path + datetime.datetime.strptime(str(start_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + "_" +
                      datetime.datetime.strptime(str(end_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + ".par"
    )
    print(start_time)
    
    
    num_cores = int(mp.cpu_count())
    pool = mp.Pool(num_cores)
    result = {}
    t1 = time.time()
    for ticker in tqdm(df['ticker'].drop_duplicates()):
        result[ticker] = (pool.apply_async(test, args=(ticker,)))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
    pool.close()
    pool.join()
    t2 = time.time()
    print("并行执行时间：", int(t2 - t1))

2018-07-01 00:00:00
2019-01-01 00:00:00
2019-07-01 00:00:00
2020-01-01 00:00:00
2020-07-01 00:00:00
2021-01-01 00:00:00
2021-07-01 00:00:00
2022-01-01 00:00:00


In [ ]:
import multiprocessing as mp
import time
import pandas as pd
from tqdm import tqdm
import datetime
from datetime import timedelta

def test(ticker):
    one_data = df[df['ticker'] == ticker]
    one_data.set_index(['timestamp','ticker'],inplace=True)
    target_df = one_data[return_value]
    one_data.drop(return_value,axis=1,inplace=True)
    concat_df = pd.DataFrame()
    for day in range(29,-1,-1):
        concat_df = pd.concat([concat_df,one_data.shift(day)],axis=1)
    concat_df.dropna(axis=0,how="any",inplace=True)
    concat_df = pd.concat([concat_df,target_df],axis=1,join='inner')
#     concat_df.to_csv(output_path + str(ticker) +".csv")
    return concat_df

def save(start_time,end_time):
    final_df[
        (final_df['timestamp'] >= start_time) &
        (final_df['timestamp'] < end_time)].set_index(['timestamp','ticker']).to_parquet(
        output_path + datetime.datetime.strptime(str(start_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + "_" +
                      datetime.datetime.strptime(str(end_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + ".par"
    )
    print(start_time)
    return None

if __name__=="__main__":
    return_value = '5d_ret'
    time_list = [20140101,20180101, 20180630, 20181231, 20190630, 20191231, 20200601, 20201231, 20210630]
    df = pd.read_csv("/home/wuwenjun/Data/AlphaNet_Original_Input.csv")
    output_path = '/home/wuwenjun/Data/AlphaNet_Original_Input/'

    num_cores = int(mp.cpu_count())
    pool = mp.Pool(num_cores)
    result = {}
    print("Start multiprocessing")
    t1 = time.time()
    for ticker in tqdm(df['ticker'].drop_duplicates()):
        result[ticker] = (pool.apply_async(test, args=(ticker,)))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
    pool.close()
    pool.join()
    t2 = time.time()
    print("running time", int(t2 - t1))
    df_list = []
    for i in tqdm(result):
        df_list.append(result[i].get())
    final_df = pd.concat(df_list)

    # preprocessing
    final_df.reset_index(inplace=True)
    final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])
    final_df.columns = [str(i) for i in range(final_df.shape[1])]
    final_df.rename(columns={'0': 'timestamp', '1': 'ticker', str(final_df.shape[1] - 1): 'target'}, inplace=True)

    # Save Data
#     num_cores = int(mp.cpu_count())
#     pool = mp.Pool(num_cores)
#     result = {}
    t1 = time.time()
    for i in range(len(time_list) - 1):
        save(start_time,end_time)
#         start_time = pd.to_datetime(str(time_list[i]))
#         end_time = pd.to_datetime(str(time_list[i + 1]))
#         pool.apply_async(save, args=(start_time,end_time))
#     pool.close()
#     pool.join()
    t2 = time.time()
    print("running time", int(t2 - t1))

100%|██████████| 4276/4276 [00:00<00:00, 22424.64it/s]
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib

/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib

/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib

/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib

...

/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib

/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib

/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib

/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/wuwenjun/.local/lib

running time 150
